<a href="https://colab.research.google.com/github/foxtrotmike/musings/blob/main/vtransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

def generate_dynamic_data(num_patients, max_time_points, num_features):
    data = []

    # Generate data for each patient
    for patient_id in range(1, num_patients + 1):
        # Randomly generate a number of time points for each patient
        num_time_points = np.random.randint(5, max_time_points + 1)

        # Generate increasingly spaced time points using a cumulative sum to ensure they are sorted
        time_increments = np.random.randint(1, 10, size=num_time_points)  # Random time intervals between observations
        times = np.cumsum(time_increments)  # Cumulative sum to ensure times are sorted

        # Initialize measurements at the first time point
        measurements = np.random.rand(num_features)  # Initial random values for each feature

        # Record initial state
        data.append([patient_id, times[0], *measurements])

        # Generate measurements for subsequent time points
        for time_idx in range(1, num_time_points):
            new_measurements = []
            time = times[time_idx]
            time_prev = times[time_idx - 1]
            time_diff = time - time_prev

            for feature_idx in range(num_features):
                # Calculate new feature value based on the given formula
                new_value = (feature_idx + 1) * measurements[feature_idx] + time_diff
                new_measurements.append(new_value)

            # Update measurements for the next iteration
            measurements = new_measurements

            # Append the new measurements to the data list
            data.append([patient_id, time, *measurements])

    # Convert data to DataFrame
    column_names = ['patient_id', 'time'] + [f'measurement_{i+1}' for i in range(num_features)]
    df = pd.DataFrame(data, columns=column_names)

    return df

# Generate a toy dataset
num_patients = 5
max_time_points = 10
num_features = 3  # Number of biomedical features
df = generate_dynamic_data(num_patients, max_time_points, num_features)

# Display the first few rows of the dataset
print(df.head(15))



import torch
import torch.nn as nn
import torch.optim as optim

class TimeSeriesTransformer(nn.Module):
    def __init__(self, feature_size, num_layers, num_heads, dropout_rate=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.transformer = nn.Transformer(
            d_model=feature_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dropout=dropout_rate,
            batch_first=True
        )
        self.output_layer = nn.Linear(feature_size, feature_size)

    def forward(self, src, tgt, src_mask, tgt_mask):
        # src and tgt are the input and target sequences
        # src_mask and tgt_mask are the padding and causal masks respectively
        out = self.transformer(src, tgt, src_key_padding_mask=src_mask, tgt_mask=tgt_mask, memory_key_padding_mask=src_mask)
        return self.output_layer(out)

def generate_square_subsequent_mask(sz):
    mask = torch.triu(torch.ones(sz, sz, device=src.device)).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

#def train(transformer, data_loader, loss_fn, optimizer, epochs=10):


# Example use case: Assume `data_loader` is your PyTorch DataLoader that yields batches of data.
# This is a minimalistic and illustrative example. In practice, you'll need to define the dataset class,
# and handle batching, and possibly GPU computations.
feature_size = 3  # Same as measurement dimensions
num_layers = 3
num_heads = 3
transformer = TimeSeriesTransformer(feature_size, num_layers, num_heads)
optimizer = optim.Adam(transformer.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Mock data loader (replace this with actual data preparation)
src = torch.rand(10, 20, feature_size)  # (batch_size, sequence_length, feature_size)
tgt = torch.rand(10, 20, feature_size)  # Same shape as src for simplicity
data_loader = [(src, tgt)]

# Train the model
transformer.train()
for epoch in range(100):
    for src, tgt in data_loader:
        src_mask = None  # No source masking in this autoregressive task
        tgt_input = tgt[:, :-1]  # Use all but the last token for input to predict the next token
        tgt_output = tgt[:, 1:]  # Use all but the first token for the target output
        tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))  # Mask for the input part

        optimizer.zero_grad()
        output = transformer(src, tgt_input, src_mask, tgt_mask)
        loss = loss_fn(output, tgt_output)
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    patient_id  time  measurement_1  measurement_2  measurement_3
0            1     3       0.650970       0.628381       0.591015
1            1     4       1.650970       2.256761       2.773046
2            1     6       3.650970       6.513522      10.319137
3            1    12       9.650970      19.027045      36.957412
4            1    15      12.650970      41.054090     113.872235
5            1    18      15.650970      85.108180     344.616704
6            2     7       0.561790       0.492265       0.254079
7            2     8       1.561790       1.984531       1.762236
8            2    13       6.561790       8.969061      10.286709
9            2    14       7.561790      18.938123      31.860126
10           2    19      12.561790      42.876245     100.580379
11           3     4       0.743339       0.254386       0.654445
12           3    10       6.743339       6.508771       7.963336
13           3    17      13.743339      20.017543      30.890008
14        

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 4, Loss: 0.5918954014778137
Epoch 5, Loss: 0.49352073669433594
Epoch 6, Loss: 0.4048081338405609
Epoch 7, Loss: 0.3490794003009796
Epoch 8, Loss: 0.3005492389202118
Epoch 9, Loss: 0.26958638429641724
Epoch 10, Loss: 0.23460052907466888
Epoch 11, Loss: 0.21801765263080597
Epoch 12, Loss: 0.19784103333950043
Epoch 13, Loss: 0.19142590463161469
Epoch 14, Loss: 0.18658648431301117
Epoch 15, Loss: 0.1781204491853714
Epoch 16, Loss: 0.1634501963853836
Epoch 17, Loss: 0.1551261693239212
Epoch 18, Loss: 0.1424313187599182
Epoch 19, Loss: 0.13384288549423218
Epoch 20, Loss: 0.1246180459856987
Epoch 21, Loss: 0.1151435449719429
Epoch 22, Loss: 0.11274189502000809
Epoch 23, Loss: 0.103813037276268
Epoch 24, Loss: 0.09800536930561066
Epoch 25, Loss: 0.10061686486005783
Epoch 26, Loss: 0.09155683219432831
Epoch 27, Loss: 0.09469671547412872
Epoch 28, Loss: 0.09001170098781586
Epoch 29, Loss: 0.08556445688009262
Epoch 30, Loss: 0.08876094222068787
Epoch 31, Loss: 0.08997109532356262
Epoch 32, 